In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import igraph
import time
import gc
from os import path
import ipython_memory_usage
import numpy as np

In [2]:
%ipython_memory_usage_start

'memory profile enabled'

In [2] used 0.1094 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 126.67 MiB


Загрузка данных

In [3]:
vertices = pd.read_csv('data\\vertices.csv') ## вершины ребер, то есть юр.лица
edges = pd.read_csv('data\\edges.csv') ## ребра графа, то есть транзакции между юр.лицами

In [3] used 259.5664 MiB RAM in 3.06s, peaked 16.11 MiB above current, total RAM usage 386.23 MiB


In [4]:
vertices = vertices.fillna(0)

In [4] used 0.1797 MiB RAM in 0.24s, peaked 116.96 MiB above current, total RAM usage 386.41 MiB


Добавим в данные о вершинах информацию о том, представлено ли юр.лицо в тестовой выборке

Закодируем данные о типе компании и ОКВЭД

In [5]:
my_encoder = LabelEncoder()
vertices['company_type']= my_encoder.fit_transform(vertices['company_type'])

In [5] used -3.0781 MiB RAM in 0.32s, peaked 13.10 MiB above current, total RAM usage 383.34 MiB


## Validation of models

1. Выбираем фирмыы, у которых более 1000 транзакций
2. Из этих фирм выбираем случайныые 100 фирм
3. У этих фирм оставляем для тренировочной выборки 500 транзакций
4. Среди всех транзакий оставленных для тестовой выборки выбираем случайные 100 000 транзакций
5. Пытаемся их предсказать

In [6]:
## данные о количестве транзакций для каждой фирмы
comp_counts = edges['id_1'].append(edges['id_2'], ignore_index = True).value_counts()

cv = []
companies_cv = []
cv_length = 3
randoms = [13, 4444, 555]

for i in range(0, cv_length):
    cur_random = randoms[i]
    ## выберем 100 случайных компаний, у которых транзакций более 1_000

##### БЫЛИ ТОЛЬКО КОМПАНИИ С БОЛЕЕ ЧЕМ 1000 ТРАНЗАКЦИЯМИ
    
    sample_comp = comp_counts[comp_counts >= 1000].sample(100, random_state = cur_random).index 
    ##Выберем случайные 500 транзакций у выбаранных фирм, чтобы оставить их для обучающей выборки
    sampled_train_edges = pd.DataFrame()
    
    for company in sample_comp:
        temp_sampled = edges[(edges['id_1'] == company) | (edges['id_2'] == company)].sample(500, random_state = cur_random)
        sampled_train_edges = sampled_train_edges.append(temp_sampled)
    
    ## выберем 100 000 случайных транзакций среди компаний, у которых более 700 транзакций, предварительно исключив 
    ## данные о транзакциях, оставленных для обучения
    temp_edges = edges[edges.index.isin(sampled_train_edges.index) == False]
    sampled_test_edges = temp_edges[temp_edges['id_1'].isin(sample_comp) | temp_edges['id_2'].isin(sample_comp)].sample(100_000, 
                                                                                                random_state = cur_random)
    train = edges[edges.index.isin(sampled_test_edges.index) == False].index.values
    test = sampled_test_edges.index.values
    cv.append((train, test))
    companies_cv.append(sample_comp)

In [6] used 355.1172 MiB RAM in 15.21s, peaked 161.73 MiB above current, total RAM usage 738.45 MiB


## Building models

In [7]:
from lightgbm import LGBMClassifier
import catboost as cat
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [7] used 2.8945 MiB RAM in 0.44s, peaked 0.00 MiB above current, total RAM usage 741.35 MiB


Для того, чтобы максимально повторить задание будем строить и оценивть модель следующим образом:\
1. Берем список фирм, для которых возможно удалены транзации(sample_comp)
2. Строим список всех возможных транзакций для нашего списка фирм
3. Удаляем из данного списка те транзакции, которые уже есть в нашей выборке
4. Предсказываем вероятность того, что данная транзакция действительно правдива

Т.к. всего у нас 1534749 фирм, то множество всех уникальных комбинаций 1534749^2. Это слишком много, поэтому будем строить модель отдельно для каждой фирмы. Тогда количество всех комбинаций будет лишь 1534749

In [8]:
vertices['is_true'] = 0
vert_id_istrue = vertices[['id', 'is_true']]
vert_no_true = vertices.drop(['is_true'], axis = 'columns')

In [8] used 144.5547 MiB RAM in 0.22s, peaked 0.00 MiB above current, total RAM usage 885.90 MiB


In [9]:
def custom_cv(model, cat_features):
    
    ## Для кэтбуста упоминание о том, что надо проверрить категориальные фичи
    if type(model).__name__ == 'CatBoostClassifier':
        print('Check catfeatures')
    
    ## Создаем пустой вектор результатов для каждого фолда
    cv_results = []
    for fold in range(0,cv_length):
        
        ## Список всех ребер для тренировки, которые действительно существуют
        train = edges[edges.index.isin(cv[fold][0])][['id_1', 'id_2']]
        ## Список всех ребер для теста, которые действительно существуют
        test = edges[edges.index.isin(cv[fold][1])][['id_1', 'id_2']]
        ## Делаем из тестовой выборки список tuples
        test = [edge[1:] for edge in test.to_records().tolist()]
        
        ## Пустой датафрейм с вероятностями для каждого ребра
        predicted_prob = pd.DataFrame()
        for firm in companies_cv[fold]:
            ## таймер для подсчета времени обучения модели для 1 фирмы
            start = time.time()
            
            ## Создаем список всех фирм, с которой компания имеет связи.
            temp_1 = train[train['id_1'] == firm]['id_2'].rename('id')
            temp_2 = train[train['id_2'] == firm]['id_1'].rename('id')
            temp = pd.DataFrame(temp_1.append(temp_2, ignore_index = True))
            temp['is_true'] = 1
            temp = temp.drop_duplicates(['id', 'is_true'])
            
            ## Добавляем в список все фирмы, с которыми компания могла бы иметь связь
            temp = temp.append(vert_id_istrue, ignore_index = True)
            temp = temp.drop_duplicates(subset = ['id'])
            temp = temp.merge(vert_no_true, on = ['id'], how = 'inner')

            ## Удаляем ненужную информацию из тренировочной выборки
            X = temp.drop(['is_true', 'id'], axis = 'columns')
            
            ## Для кэтбуста делаем кэтфичи
            if cat_features == True:
                X['main_okved'] = X['main_okved'].astype(str)  ## делаем строку
            
            ## target value
            y = temp['is_true']
            
            
            model_trained = model.fit(X, y)
            proba = pd.DataFrame(model_trained.predict_proba(X))
            
            ## Создаем датасет с предсказанными ребрами
            proba['id_1'] = firm
            proba['id_2'] = temp['id'].values
            proba['is_true'] = y.values
            
            ## Прикрепляем в общий датасет с предсказанными вершинами
            predicted_prob = predicted_prob.append(proba)
            print(firm, fold, time.time() - start)
            
        
        del temp_1, temp_2, temp, X, y, train
        gc.collect()
        
        ## Удаляем вершины, кооторые уже были в тренировочной выборке
        predicted_prob = predicted_prob[predicted_prob['is_true'] != 1]
        
        ## Сортируем вершины по вероятности правдивости
        predicted_prob = predicted_prob.sort_values([1], ascending = False)[['id_1', 'id_2']]
        
        ## Делаем из предсказанных вершин список tuples
        predicted_prob = [edge[1:] for edge in predicted_prob.to_records().tolist()]
        
        ## Выбираем 100_000 самых вероятных вершин, при этом удаляя повторяющиеся
        print('Selecting top 100_000')
        predicted_edges = []
        
        for edge in predicted_prob:
            if (edge not in predicted_edges) & (tuple([edge[place] for place in [1,0]]) not in predicted_edges) &\
            (len(predicted_edges) <= 99_999):
                predicted_edges.append(edge)
            
            elif (len(predicted_edges) > 99_999):
                break
        
        ## Подсчитываем, сколько из 100_000 предсказанных вершин действительно есть в тестовой выборке  
        print('scoring')
        score = 0
        
        for predicted_edge in predicted_edges:
            if (predicted_edge in test) | (tuple([predicted_edge[place] for place in [1,0]]) in test):
                score = score + 1
        
        del predicted_prob, test, predicted_edges
        gc.collect()
        
        cv_results.append(score)
        print(score)
        del score
        gc.collect()
    return(cv_results)

In [9] used 0.3047 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 886.21 MiB


In [10]:
def final_results(cv, models_to_use, description, model_cols, cat_features = False):
    names = []
    params = []
    scores = []
    description = [description] * len(models_to_use)
    model_cols = [model_cols] * len(models_to_use)
    
    for model in models_to_use:
        model_score = custom_cv(model, cat_features)
        model_score.append(np.mean(model_score))
        scores.append(model_score)
        
        names.append(type(model).__name__)
        params.append(model.get_params())
        
    final_results = pd.DataFrame({'model_name' : names, 'model_params' : params, 
                                  'model_description' : description, 'model_cols' : model_cols})
    
    for cv_split in range(1, np.array(scores).shape[1]+1):
        final_results[str(cv_split)] = np.array(scores).T[cv_split-1]
        
    final_results['mean'] = final_results.iloc[:, -1]
    final_results = final_results.drop([final_results.columns[-2]], axis = 'columns')
    
    if path.exists("final_results.csv"):
        final_results_loaded = pd.read_csv('final_results.csv')
        final_results = final_results_loaded.append(final_results, ignore_index = True)    
    final_results.to_csv('final_results.csv', index = False)

In [10] used 0.0547 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 886.26 MiB


In [ ]:
final_results(cv, 
             [LogisticRegression(random_state = 0, n_jobs = -1)],
              'Full graph. As in baseline. Mean, std and sum of n_tran and value for every firm. Filled nans with 0',
             ['main_okved', 'region_code', 'company_type'],
             cat_features = False)

In [ ]:
final_results(cv, 
             [LGBMClassifier(n_estimators = 100, random_state = 0, n_jobs = -1),
              LGBMClassifier(n_estimators = 500, random_state = 0, n_jobs = -1)],
              'Full graph. As in baseline. Mean, std and sum of n_tran and value for every firm. Filled nans with 0',
             ['main_okved', 'region_code', 'company_type'],
             cat_features = False)

In [ ]:
final_results(cv, 
             [CatBoostClassifier(n_estimators = 500, cat_features= ['main_okved', 'region_code', 'company_type'], 
                                 random_state = 0, verbose = False, task_type = 'GPU')],
              'Full graph. As in baseline. Mean, std and sum of n_tran and value for every firm. Filled nans with 0',
             ['main_okved', 'region_code', 'company_type'],
             cat_features = True)

In [3]:
res = pd.read_csv('final_results.csv')

In [9]:
res.to_csv('final_results.csv', index = False)

In [8]:
res

,model_name,model_params,model_description,model_cols,1,2,3,mean,results_leaderboard
0,LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",4351.0,4345.0,4158.0,4284.666667,1126.0
1,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",631.0,1130.0,663.0,808.000000,NaN
2,CatBoostClassifier,"{'iterations': 100, 'verbose': False, 'cat_fea...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",5765.0,5746.0,5724.0,5745.000000,4675.0
3,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",3481.0,3535.0,3266.0,3427.333333,NaN
4,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",19.0,83.0,40.0,47.333333,NaN
5,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",3868.0,4566.0,3822.0,4085.333333,NaN
6,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",5794.0,6073.0,5148.0,5671.666667,2149.0
7,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",2320.0,2199.0,2212.0,2243.666667,NaN
8,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",2343.0,2425.0,2559.0,2442.333333,NaN
9,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",Full graph. As in baseline,"['main_okved', 'region_code', 'company_type']",5831.0,5649.0,5351.0,5610.333333,NaN
